# Pre-processing Arizona Allocation data for WaDEQA upload.
Date Updated: 04/06/2020
Purpose:  To pre-process the Arizona data into one master file for simple DataFrame creation and extraction

Notes:
We are going to all AZ groundwater for now.  They don't tradtionaly track priority date, but we want to incldue the volume and site info.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

## Groundwater

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData/Groundwater"
os.chdir(workingDir)

WellRegistry_Input = "WELLS_wellRegistry_input.csv"
GWSI_Input = "GWSI_SITES_input.csv"

In [3]:
#Groundwater Dataframes creation
df_WR = pd.read_csv(WellRegistry_Input)
df_WR['REG_ID'] = df_WR['REGISTRY_I'].astype(str)
df_GWSI = pd.read_csv(GWSI_Input)

# Merge together into one
df_WR_GWSI = pd.merge(df_WR, df_GWSI, left_on='REG_ID', right_on='REG_ID', how='left')
df_WR_GWSI.head(3)

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,31,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,OID__x,PROGRAM,REGISTRY_I,OWNER_NAME,RGR_PUMP_D,WELLTYPE,WELL_TYPE_,DLIC_NUM,APPROVED,INSTALLED,WELL_DEPTH_x,WATER_LEVE,CASING_DEP,CASING_DIA,CASING_TYP,PUMP_TYPE,PUMP_POWER,PUMPRATE,TESTEDRATE,DRAW_DOWN,COMPLETION,DRILL_LOG,WELL_CANCE,CADASTRAL,COUNTY,WATERSHED,BASIN_NAME,SUBBASIN_N,AMA,QUAD_CODE,WHOLE_TOWN,HALF_TOWNS,NORTHSOUTH,WHOLE_RANG,HALF_RANGE,EASTWEST,SECTION,QUARTER_16,QACRE160DI,QUARTER_40,QACRE40DIR,QUARTER_10,QACRE10DIR,UTM_X_METE,UTM_Y_METE,APPLICATIO,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,ZIP4,REG_ID,OID__y,OBJECTID,SITE_ID,LOCAL_ID,WELL_TYPE,DD_LAT,DD_LONG,WELL_ALT,WATER_USE,WELL_DEPTH_y,CASE_DIAM,LASTWLDATE,WL_DTW,WL_ELEV
0,0,55,60000,AR SACATON LLC,NO,NON-EXEMPT,NON-EXEMPT,0,NaN,NaN,1790,257.0,60,20.0,,,NO POWER CODE LISTED,0,0,0,,,Y,D05005035ABA,PINAL,SANTA CRUZ RIVER,PINAL AMA,ELOY,PINAL,D,5,0,S,5,0,E,35,A,NE,B,NW,A,NE,423912.1,3646244.0,1/1/1900 0:00:00,1150 N 7TH AVE,PO BOX 5747,TUCSON,AZ,,,60000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,1/24/2003 0:00:00,7/2/1984 0:00:00,1000,520.0,1000,12.0,,,ELECTRIC MOTOR 16 - 100 HP,200,400,75,,,,B05002008CCB,MARICOPA,AGUA FRIA RIVER,PHOENIX AMA,WEST SALT RIVER VALLEY,PHOENIX,B,5,0,N,2,0,W,8,C,SW,C,SW,B,NW,361297.1,3739323.0,2/25/1985 0:00:00,"1114 AVENUE OF THE AMERICAS, 39TH FLOOR",,NEW YORK,NY,,,60001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,55,82721,"KEYES,G",NO,NON-EXEMPT,NON-EXEMPT,63,NaN,NaN,0,0.0,0,0.0,,,NO POWER CODE LISTED,0,0,0,,,Y,B02009022000,MARICOPA,LOWER GILA RIVER,HARQUAHALA INA,HARQUAHALA,HARQUAHALA INA,B,2,0,N,9,0,W,22,0,,0,,0,,298366.4,3708927.0,10/15/1979 0:00:00,7765 E FOOTHILL DR S,,PARADISE VALLEY,AZ,,,82721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Creating long and lat values from data.  
# Need to convert from UTM to WGS 84.
# I believe AZ is consiered WGS 84 / UTM zone 12N - EPSG:32612.

from pyproj import Proj
myProj = Proj(proj='utm',zone=12, ellps='WGS84', preserve_units=False)
long, lat = myProj(df_WR_GWSI['UTM_X_METE'].values, df_WR_GWSI['UTM_Y_METE'].values, inverse=True)
df_WR_GWSI['in_Latitude'] = lat
df_WR_GWSI['in_Longitude'] = long
df_WR_GWSI = df_WR_GWSI.replace(np.nan, '')  # Replaces NaN values with blank.
df_WR_GWSI.head(3)

,OID__x,PROGRAM,REGISTRY_I,OWNER_NAME,RGR_PUMP_D,WELLTYPE,WELL_TYPE_,DLIC_NUM,APPROVED,INSTALLED,WELL_DEPTH_x,WATER_LEVE,CASING_DEP,CASING_DIA,CASING_TYP,PUMP_TYPE,PUMP_POWER,PUMPRATE,TESTEDRATE,DRAW_DOWN,COMPLETION,DRILL_LOG,WELL_CANCE,CADASTRAL,COUNTY,WATERSHED,BASIN_NAME,SUBBASIN_N,AMA,QUAD_CODE,WHOLE_TOWN,HALF_TOWNS,NORTHSOUTH,WHOLE_RANG,HALF_RANGE,EASTWEST,SECTION,QUARTER_16,QACRE160DI,QUARTER_40,QACRE40DIR,QUARTER_10,QACRE10DIR,UTM_X_METE,UTM_Y_METE,APPLICATIO,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,ZIP4,REG_ID,OID__y,OBJECTID,SITE_ID,LOCAL_ID,WELL_TYPE,DD_LAT,DD_LONG,WELL_ALT,WATER_USE,WELL_DEPTH_y,CASE_DIAM,LASTWLDATE,WL_DTW,WL_ELEV,in_Latitude,in_Longitude
0,0,55,60000,AR SACATON LLC,NO,NON-EXEMPT,NON-EXEMPT,0,,,1790,257.0,60,20.0,,,NO POWER CODE LISTED,0,0,0,,,Y,D05005035ABA,PINAL,SANTA CRUZ RIVER,PINAL AMA,ELOY,PINAL,D,5,0,S,5,0,E,35,A,NE,B,NW,A,NE,423912.1,3646244.0,1/1/1900 0:00:00,1150 N 7TH AVE,PO BOX 5747,TUCSON,AZ,,,60000,,,,,,,,,,,,,,,32.951858,-111.814054
1,1,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,1/24/2003 0:00:00,7/2/1984 0:00:00,1000,520.0,1000,12.0,,,ELECTRIC MOTOR 16 - 100 HP,200,400,75,,,,B05002008CCB,MARICOPA,AGUA FRIA RIVER,PHOENIX AMA,WEST SALT RIVER VALLEY,PHOENIX,B,5,0,N,2,0,W,8,C,SW,C,SW,B,NW,361297.1,3739323.0,2/25/1985 0:00:00,"1114 AVENUE OF THE AMERICAS, 39TH FLOOR",,NEW YORK,NY,,,60001,,,,,,,,,,,,,,,33.784972,-112.498131
2,2,55,82721,"KEYES,G",NO,NON-EXEMPT,NON-EXEMPT,63,,,0,0.0,0,0.0,,,NO POWER CODE LISTED,0,0,0,,,Y,B02009022000,MARICOPA,LOWER GILA RIVER,HARQUAHALA INA,HARQUAHALA,HARQUAHALA INA,B,2,0,N,9,0,W,22,0,,0,,0,,298366.4,3708927.0,10/15/1979 0:00:00,7765 E FOOTHILL DR S,,PARADISE VALLEY,AZ,,,82721,,,,,,,,,,,,,,,33.500894,-113.170597


In [5]:
# Create output dataframe for ground water
columnslist = [
    ### Method Info ###
    "in_ApplicableResourceTypeCV",
    
    ### Water Source Info ###
    "in_WaterSourceUUID",
    "in_WSGeometry",
    "in_GNISFeatureNameCV",
    "in_WaterQualityIndicatorCV",
    "in_WaterSourceName",
    "in_WaterSourceNativeID",
    "in_WaterSourceTypeCV",
    
    ### Site Info ###
    "in_SiteUUID",
    "in_CoordinateAccuracy",
    "in_CoordinateMethodCV",
    "in_County",
    "in_EPSGCodeCV",
    "in_Geometry",
    "in_GNISCodeCV",
    "in_HUC12",
    "in_HUC8",
    "in_Latitude",
    "in_Longitude",
    "in_NHDNetworkStatusCV",
    "in_NHDProductCV",
    "in_PODorPOUSite",
    "in_SiteName",
    "in_SiteNativeID",
    "in_SitePoint",
    "in_SiteTypeCV",
    "in_StateCV",
    "in_USGSSiteID",
    
    ### AllocationAmount_fact Info ###
    "in_AllocationApplicationDate",
    "in_AllocationAssociatedConsumptiveUseSiteIDs",
    "in_AllocationAssociatedWithdrawalSiteIDs",
    "in_AllocationBasisCV",
    "in_AllocationChangeApplicationIndicator",
    "in_AllocationCommunityWaterSupplySystem",
    "in_AllocationCropDutyAmount",
    "in_AllocationExpirationDate",
    "in_AllocationFlow_CFS",
    "in_AllocationLegalStatusCV",
    "in_AllocationNativeID",
    "in_AllocationOwner",
    "in_AllocationPriorityDate",
    "in_AllocationSDWISIdentifierCV",
    "in_AllocationTimeframeEnd",
    "in_AllocationTimeframeStart",
    "in_AllocationTypeCV",
    "in_AllocationVolume_AF",
    "in_BeneficialUseCategory",
    "in_CommunityWaterSupplySystem",
    "in_CropTypeCV",
    "in_CustomerTypeCV",
    "in_DataPublicationDate",
    "in_DataPublicationDOI",
    "in_ExemptOfVolumeFlowPriority",
    "in_GeneratedPowerCapacityMW",
    "in_IrrigatedAcreage",
    "in_IrrigationMethodCV",
    "in_LegacyAllocationIDs",
    "in_MethodUUID",
    "in_OrganizationUUID",
    "in_PopulationServed",
    "in_PowerType",
    "in_PrimaryUseCategory",
    "in_SiteUUID",
    "in_VariableSpecificUUID",
    "in_WaterAllocationNativeURL",
    "in_WaterSourceUUID"
]

dfground = pd.DataFrame(columns=columnslist, index=df_WR_GWSI.index)


In [6]:
#############################################################################################
#Method
dfground['in_ApplicableResourceTypeCV'] = 'Groundwater'

#WaterSource
dfground['in_WaterSourceTypeCV'] = 'groundwater/well'
dfground['in_WaterSourceName'] = "Unspecified"
                                    
#Site
dfground['in_County'] = df_WR_GWSI['COUNTY']
dfground['in_Latitude'] = df_WR_GWSI['in_Latitude']
dfground['in_Longitude'] = df_WR_GWSI['in_Longitude']
dfground['in_PODorPOUSite'] = 'POD'
dfground['in_SiteName'] = "Groundwater"
dfground['in_SiteNativeID'] = df_WR_GWSI['REGISTRY_I']
dfground['in_SiteTypeCV'] = df_WR_GWSI['WELL_TYPE_']

#AllocationAmount_fact
dfground['in_AllocationFlow_CFS'] = df_WR_GWSI['PUMPRATE']
dfground['in_AllocationNativeID'] = df_WR_GWSI['REGISTRY_I']
dfground['in_AllocationOwner'] = df_WR_GWSI['OWNER_NAME']
dfground['in_AllocationPriorityDate'] = ''
dfground['in_AllocationTimeframeEnd'] = '12/31'
dfground['in_AllocationTimeframeStart'] = '01/01'
dfground['in_AllocationTypeCV'] = df_WR_GWSI['WELL_TYPE_']
dfground['in_BeneficialUseCategory'] = df_WR_GWSI['WATER_USE']
dfground['in_ExemptOfVolumeFlowPriority'] = 1

In [7]:
# Drop Duplicates
dfground = dfground.drop_duplicates()
dfground = dfground.replace(np.nan, '')  # Replaces NaN values with blank.

## Surface Water

In [8]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData/Surface_Water"
os.chdir(workingDir)

#Surfacewater Dataframes creation
SWR_fillings_input = "SWR_fillings_POD_input.csv"
df_SWRfill = pd.read_csv(SWR_fillings_input, encoding = "ISO-8859-1")

SWshed_input = "ADWR_Surface_Watersheds_POD_input.csv"
df_SWshed = pd.read_csv(SWshed_input, encoding = "ISO-8859-1")

In [9]:
# Need to restructure df_SWshed
# Single Flow_CFS as ANNUAL USE value.
# Seperate, then combine via comma Water Use.
# Retreive BenUse to single flow df.

df_SWshed_AU = df_SWshed.loc[df_SWshed["WATER USE"] == "ANNUAL USE"].reset_index()
df_SWshed_Ben = df_SWshed.loc[df_SWshed["WATER USE"] != "ANNUAL USE"].reset_index()

df_SWshed_Ben = df_SWshed_Ben.groupby('REG. NO').agg(lambda x: ','.join([str(elem) for elem in (list(set(x)))])).replace(np.nan, '').reset_index()
df_SWshed_Ben['BenUse'] = df_SWshed_Ben['WATER USE']
df_SWshed_Ben['REGNO'] = df_SWshed_Ben['REG. NO']

BenUseDict = pd.Series(df_SWshed_Ben.BenUse.values, index = df_SWshed_Ben.REGNO).to_dict()
def retrieveBenUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue
        try:
            outList = BenUseDict[String1]
        except:
            outList = ''
    return outList
df_SWshed_AU['BenUse'] = df_SWshed_AU.apply(lambda row: retrieveBenUse(row['REG. NO']), axis=1)

In [10]:
#Merge together
df_SWFill_SWshed = pd.merge(df_SWRfill, df_SWshed_AU, left_on='FILE_NO', right_on='REG. NO', how='left')

In [11]:
# Creating long and lat values from data.  
# Need to convert from UTM 12N to WGS 84.
# I believe AZ is consiered WGS 84 / UTM zone 12N - EPSG:32612.

from pyproj import Proj
myProj = Proj(proj='utm',zone=12, ellps='WGS84', preserve_units=False)
long, lat = myProj(df_SWFill_SWshed['X_UTMNAD83'].values, df_SWFill_SWshed['Y_UTMNAD83'].values, inverse=True)
df_SWFill_SWshed['in_Latitude'] = lat
df_SWFill_SWshed['in_Longitude'] = long
df_SWFill_SWshed = df_SWFill_SWshed.replace(np.nan, '')  # Replaces NaN values with blank.
df_SWFill_SWshed.head(3)

,ï»¿OID_,PROGRAM,APPNO,CONVNO,CERTNO,CERTSUFX,PERMITNO,PERMITSUF,PARENTAPP,POU_POD,STATUS_x,WATERSOURC,FILE_DATE,PRIOR_DATE,CONST_DT,USESTAT,RSRVNAME,HLDRNAME,HLDRCONAME,HLDRADDR,HLDRADDR2,HLDRCITY,HLDRZIP,HLDRZIP2,USES,WS_DESCR,CADASTRAL,CADASTRAL_,X_UTMNAD83,Y_UTMNAD83,FILE_NO,index,REG. NO,NAME,ART_WS_IDNO_FKFLD,ADDRESS,REG. NO.1,STATUS_y,PERMIT NO,CERT. NO,FILE DATE,SOURCE,PRIOR DATE,COUNTY,OWNER TYPE,WATERSHED,LEGAL,POU/POD,WATER USE,QUANTITY,BenUse,in_Latitude,in_Longitude
0,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,1146,38-94570.0,AZ STATE LAND DEPT,36,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033,1.5 DRAW,25203,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,"STOCK,WILDLIFE",32.666443,-110.864362
1,170705,36,5457,0,,,,,36-5457,POD,ACTIVE - ACTIVE,1.5,9/30/1988,12/31/1883,,,,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,COWS / HORSES for STOCK for WILDLIFE using .4...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,36-5457.0,1143,36-5457.0,AZ STATE LAND DEPT,36,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",36-5457.0,ACTIVE - ACTIVE,,,32416,1.5 DRAW,-5843,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,"STOCK,WILDLIFE",32.666443,-110.864362
2,12726,38,94943,0,,,,,,POD,ACTIVE - ACTIVE,3.5,6/27/1979,12/31/1957,12/31/1957 0:00,,3.5 SPREADER TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE13 for STOCK,WILLCOX PLAYA,D15027030AC0,D15027030AC0,629575.9955,3552696.603,38-94943.0,,,,,,,,,,,,,,,,,,,,,32.103168,-109.626695


In [12]:
# Extracting Flow CFS Values from string

def CreateFlow_CFS(ColRowVal):
    val = str(ColRowVal)
    try:
        val2 = val.split(' Acre')[0]
        val3 = float(val2)
    except:
        val3 = ""
    outlist = val3
    return outlist

df_SWFill_SWshed['QUANTITY'] = df_SWFill_SWshed.apply(lambda row: CreateFlow_CFS(row['QUANTITY']), axis=1)

In [13]:
# Create output dataframe for ground water
columnslist = [
    ### Method Info ###
    "in_ApplicableResourceTypeCV",
    
    ### Water Source Info ###
    "in_WaterSourceUUID",
    "in_WSGeometry",
    "in_GNISFeatureNameCV",
    "in_WaterQualityIndicatorCV",
    "in_WaterSourceName",
    "in_WaterSourceNativeID",
    "in_WaterSourceTypeCV",
    
    ### Site Info ###
    "in_SiteUUID",
    "in_CoordinateAccuracy",
    "in_CoordinateMethodCV",
    "in_County",
    "in_EPSGCodeCV",
    "in_Geometry",
    "in_GNISCodeCV",
    "in_HUC12",
    "in_HUC8",
    "in_Latitude",
    "in_Longitude",
    "in_NHDNetworkStatusCV",
    "in_NHDProductCV",
    "in_PODorPOUSite",
    "in_SiteName",
    "in_SiteNativeID",
    "in_SitePoint",
    "in_SiteTypeCV",
    "in_StateCV",
    "in_USGSSiteID",
    
    ### AllocationAmount_fact Info ###
    "in_AllocationApplicationDate",
    "in_AllocationAssociatedConsumptiveUseSiteIDs",
    "in_AllocationAssociatedWithdrawalSiteIDs",
    "in_AllocationBasisCV",
    "in_AllocationChangeApplicationIndicator",
    "in_AllocationCommunityWaterSupplySystem",
    "in_AllocationCropDutyAmount",
    "in_AllocationExpirationDate",
    "in_AllocationFlow_CFS",
    "in_AllocationLegalStatusCV",
    "in_AllocationNativeID",
    "in_AllocationOwner",
    "in_AllocationPriorityDate",
    "in_AllocationSDWISIdentifierCV",
    "in_AllocationTimeframeEnd",
    "in_AllocationTimeframeStart",
    "in_AllocationTypeCV",
    "in_AllocationVolume_AF",
    "in_BeneficialUseCategory",
    "in_CommunityWaterSupplySystem",
    "in_CropTypeCV",
    "in_CustomerTypeCV",
    "in_DataPublicationDate",
    "in_DataPublicationDOI",
    "in_ExemptOfVolumeFlowPriority",
    "in_GeneratedPowerCapacityMW",
    "in_IrrigatedAcreage",
    "in_IrrigationMethodCV",
    "in_LegacyAllocationIDs",
    "in_MethodUUID",
    "in_OrganizationUUID",
    "in_PopulationServed",
    "in_PowerType",
    "in_PrimaryUseCategory",
    "in_SiteUUID",
    "in_VariableSpecificUUID",
    "in_WaterAllocationNativeURL",
    "in_WaterSourceUUID"
]

df_Surface = pd.DataFrame(columns=columnslist, index=df_SWFill_SWshed.index)

In [14]:
# For creating WaterSourceName
def assignWaterSourceName(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        strVal = str(colrowValue)
        outList = strVal.strip()
    return outList

# For creating SiteNativeID
def assignSiteNativeID(colrowValue):
    strVal = str(colrowValue)
    strVal = strVal.strip()
    if strVal == '' or pd.isnull(strVal):
        outList = "Unspecified"
    else:
        outList = strVal
    return outList

#############################################################################################
#Method
df_Surface['in_ApplicableResourceTypeCV'] = 'Surface Water'

#WaterSource
df_Surface['in_WaterSourceTypeCV'] = 'Surface Water'
df_Surface['in_WaterSourceName'] = df_SWFill_SWshed.apply(lambda row: assignWaterSourceName(row['WATERSOURC']), axis=1)
                                    
#Site
df_Surface['in_County'] = df_SWFill_SWshed['COUNTY']
df_Surface['in_Latitude'] = df_SWFill_SWshed['in_Latitude']
df_Surface['in_Longitude'] = df_SWFill_SWshed['in_Longitude']
df_Surface['in_PODorPOUSite'] = "POD"
df_Surface['in_SiteName'] = "Surface Water"
df_Surface['in_SiteNativeID'] = df_SWFill_SWshed.apply(lambda row: assignSiteNativeID(row['CADASTRAL']), axis=1)
df_Surface['in_SiteTypeCV'] = "Unspecified"

#AllocationAmount_fact
df_Surface['in_AllocationFlow_CFS'] = df_SWFill_SWshed['QUANTITY']
df_Surface['in_AllocationLegalStatusCV'] = df_SWFill_SWshed['STATUS_x']
df_Surface['in_AllocationNativeID'] = df_SWFill_SWshed['FILE_NO']
df_Surface['in_AllocationOwner'] = df_SWFill_SWshed['HLDRNAME']
df_Surface['in_AllocationTimeframeEnd'] = '12/31'
df_Surface['in_AllocationTimeframeStart'] = '01/01'
df_Surface['in_AllocationPriorityDate'] = df_SWFill_SWshed['PRIOR_DATE']
df_Surface['in_BeneficialUseCategory'] = df_SWFill_SWshed['BenUse']
df_Surface['in_ExemptOfVolumeFlowPriority'] = 0

In [15]:
# Drop Duplicates
df_Surface = df_Surface.drop_duplicates()

## Concatenate and Export Outputs

In [16]:
# Concatenate
frames = [dfground, df_Surface]
dfout = pd.concat(frames)

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

in_ApplicableResourceTypeCV                      object
in_WaterSourceUUID                               object
in_WSGeometry                                    object
in_GNISFeatureNameCV                             object
in_WaterQualityIndicatorCV                       object
in_WaterSourceName                               object
in_WaterSourceNativeID                           object
in_WaterSourceTypeCV                             object
in_SiteUUID                                      object
in_CoordinateAccuracy                            object
in_CoordinateMethodCV                            object
in_County                                        object
in_EPSGCodeCV                                    object
in_Geometry                                      object
in_GNISCodeCV                                    object
in_HUC12                                         object
in_HUC8                                          object
in_Latitude                                     

In [18]:
#Exporting to Finished File
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData"
os.chdir(workingDir)

dfout.to_csv('P_ArizonaMaster.csv', index=False)  # The output